# Предсказание потребления электроэнергии в зависимости от плана производства

## Загрузка библиотек

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from sklearn.metrics import mean_absolute_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression

## Вспомогательные функции

`mape` - функция для расчета средней абсолютной ошибки в процентах.

In [ ]:
def mape(y_target, y_pred):
    return np.mean(np.abs(y_target - y_pred) / y_target)

`pv_fact_plan` - функция для создания столбца с фактическим (для обучения) и плановым (для предсказания) объемом производства, где: 
- `n_pred` - количество месяцев для предсказания.

In [ ]:
def pv_fact_plan(X, n_pred):
    pv_fact = X[['pv_fact']].iloc[: -n_pred].rename(columns={'pv_fact': 'pv'})
    pv_plan = X[['pv_plan']].iloc[-n_pred :].rename(columns={'pv_plan': 'pv'})
    
    return pd.concat([pv_fact, pv_plan]).join(features, how='left').astype('int').drop(columns=['pv_fact', 'pv_plan'])

`data_split` - функция для разделения данных на обучающую и предсказательную (тестовую) выборки, где:
- `n_pred` - количество месяцев для предсказания;
- `n_train` - количество  месяцев для обучения.

In [ ]:
def data_split(X, y, n_pred, n_train = 24):
    # обучающая выборка размером 'n_train' с учетом сдвига на величину 'n_pred'     
    X_train = X.iloc[-(n_pred + n_train): -n_pred]
    y_train = y.iloc[-(n_pred + n_train): -n_pred]
    
    # предсказательная (тестовая) выборка размером 'n_pred'     
    X_pred = X.iloc[-n_pred:]
    y_pred = y.iloc[-n_pred:]
    
    return X_train, y_train, X_pred, y_pred

`consumption_predict` - мульти-регрессионная функция для предсказания общего и фабричного потребления электроэнергии, где:
- `n_pred` - количество месяцев для предсказания:
- `only_pv_plan` - способ формирования столбца с объемом производства (`True` - использовать только плановые значения, `False` - использовать для обучения фактические значения, для предсказания плановые).      

In [ ]:
def consumption_predict(model, X, y, n_pred, only_pv_plan):
    if only_pv_plan:
        X_mod = X.drop(columns='pv_fact')   
    else:
        X_mod = pv_fact_plan(X, n_pred)
    
    X_train, y_train, X_pred, y_pred = data_split(X_mod, y, n_pred)
    regr = MultiOutputRegressor(model).fit(X_train, y_train)    
    preds = regr.predict(X_pred).astype(int)
    
    return pd.DataFrame(data=preds, columns=['Общее', 'Вафельное'], index=list(X_pred.index))

## Загрузка и подготовка данных

In [ ]:
data = 'electricity_consumption.xlsx'
df = pd.read_excel(data, sheet_name='data', index_col='period', parse_dates=True)

df.index = df.index.to_period('M')

In [ ]:
features_labels = ['pv_fact', 'pv_plan', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 
                   'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12']

target_labels = ['total','waffle']

features = df[features_labels]

target = df[target_labels]

## Предсказание

In [ ]:
LR = LinearRegression()

In [ ]:
predictions = consumption_predict(model=LR, X=features, y=target, n_pred=13, only_pv_plan=True)
predictions

,Общее,Вафельное
2022-12,631142,27415
2023-01,611802,17760
2023-02,733510,26274
2023-03,1101003,31824
2023-04,1248416,40878
2023-05,1443844,47102
2023-06,1385450,73336
2023-07,1259056,42943
2023-08,994330,44888
2023-09,953690,40068
